In [2]:
!pip install tensorflow

     |████████████████████████████████| 497.6 MB 21 kB/s s eta 0:00:01     |████████▋                       | 134.3 MB 9.6 MB/s eta 0:00:39
     |████████████████████████████████| 14.5 MB 9.6 MB/s eta 0:00:01
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
     |████████████████████████████████| 2.1 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 9.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 8.9 MB/s eta 0:00:01
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 5.8 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 9.4 MB/s eta 0:00:01
  Using cached termco

In [4]:
!pip install opencv-python

  Using cached opencv_python-4.5.5.62-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.4 MB)


In [2]:
# 학습
# Data : ALL

import tensorflow as tf
import h5py
import numpy as np
import os
from PIL import Image
import cv2

print(os.getcwd())


2022-02-08 13:54:08.653972: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-08 13:54:08.654027: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/vividlee/Desktop/YeloCar


In [5]:
dataset_version = 'deep4pc_test1'
model_version = 'model_' + dataset_version

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

# model = tf.keras.models.load_model('./script/Model/'+model_version)
# print(model.evaluate(x_test, y_test))
# model.summary()

# model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)
# outputs = model(x_test[0:10])

# print(outputs)

# print(x_test[0:10])

2022-01-18 16:46:59.418362: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-18 16:46:59.418479: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-01-18 16:46:59.418545: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-01-18 16:46:59.445869: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-01-18 16:46:59.446023: W tensorflow/stream_executor/platform/default/dso_loader

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 6, 32)         0

2022-01-18 16:54:42.334409: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./Model/Model_deep4pc_test1/model_deep4pc_test1/assets


## dataset : 'deep4pc_test1'
### Epoch 12/100 673/673 [==============================] - 113s 168ms/step - loss: 0.0038 - mae: 0.0413 - mse: 0.0038 - val_loss: 0.0123 - val_mae: 0.0777 - val_mse: 0.0123

In [1]:
dataset_version = 'deep4pc_test1'
model_version = 'model_' + dataset_version + '_2'

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

# model = tf.keras.models.load_model('./script/Model/'+model_version)
# print(model.evaluate(x_test, y_test))
# model.summary()

# model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)
# outputs = model(x_test[0:10])

# print(outputs)

# print(x_test[0:10])

NameError: name 'h5py' is not defined

In [11]:
from PIL import Image
import cv2

In [28]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf)

model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)

x_tested = x_test[0:10]
# print(type(x_test[0:10]))
# print(x_test[0:10].shape)
# print(type(x_tested))
# print(x_tested.shape)

outputs = model(x_tested)

print(outputs)

x_tested *= 255

for i in range (0, 10) :
    cv2.imwrite('filename_' + str(i) + '.jpeg', x_tested[i])


tf.Tensor(
[[0.4722335  0.1559705 ]
 [0.47223353 0.15597047]
 [0.47223586 0.15597549]
 [0.47223628 0.15597016]
 [0.4722375  0.15596777]
 [0.4722398  0.15596272]
 [0.4722401  0.15596005]
 [0.47224066 0.15596175]
 [0.472234   0.15597227]
 [0.4722414  0.15596355]], shape=(10, 2), dtype=float32)


In [29]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [31]:
dataset_version = 'deep4pc_test1'

model_version = 'model_' + dataset_version + '_3_patience10'

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)
#model.summary()
outputs = model(x_test[0:10])

print(outputs)

# x_test *= 255

createFolder('./Test/' + model_version)    
for i in range (0, 10) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 23, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 10, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 6, 32)        

2022-01-18 18:40:03.627758: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6347022336 exceeds 10% of free system memory.


Epoch 1/100
673/673 [==============================] - 121s 179ms/step - loss: 0.0079 - mae: 0.0583 - mse: 0.0079 - val_loss: 0.0122 - val_mae: 0.0776 - val_mse: 0.0122
Epoch 2/100
673/673 [==============================] - 113s 168ms/step - loss: 0.0061 - mae: 0.0506 - mse: 0.0061 - val_loss: 0.0118 - val_mae: 0.0766 - val_mse: 0.0118
Epoch 3/100
673/673 [==============================] - 113s 168ms/step - loss: 0.0056 - mae: 0.0488 - mse: 0.0056 - val_loss: 0.0132 - val_mae: 0.0802 - val_mse: 0.0132
Epoch 4/100
673/673 [==============================] - 114s 170ms/step - loss: 0.0052 - mae: 0.0475 - mse: 0.0052 - val_loss: 0.0129 - val_mae: 0.0785 - val_mse: 0.0129
Epoch 5/100
673/673 [==============================] - 113s 169ms/step - loss: 0.0049 - mae: 0.0462 - mse: 0.0049 - val_loss: 0.0126 - val_mae: 0.0771 - val_mse: 0.0126
Epoch 6/100
673/673 [==============================] - 113s 168ms/step - loss: 0.0047 - mae: 0.0452 - mse: 0.0047 - val_loss: 0.0119 - val_mae: 0.0750 - va

OSError: No file or directory found at ./script/Model/model_deep4pc_test1_3_patience15

In [39]:
dataset_version = 'deep4pc_test1'
model_version = 'model_' + dataset_version + '_4_patience15'

createFolder('./Test/' + model_version)
f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)
#model.summary()
outputs = model(x_test[0:10])

print(outputs)
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 10) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 23, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 10, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 4, 6, 32)        

2022-01-18 19:34:06.316865: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6347022336 exceeds 10% of free system memory.


Epoch 1/100
673/673 [==============================] - 114s 169ms/step - loss: 0.0080 - mae: 0.0591 - mse: 0.0080 - val_loss: 0.0113 - val_mae: 0.0724 - val_mse: 0.0113
Epoch 2/100
673/673 [==============================] - 115s 170ms/step - loss: 0.0061 - mae: 0.0508 - mse: 0.0061 - val_loss: 0.0121 - val_mae: 0.0744 - val_mse: 0.0121
Epoch 3/100
673/673 [==============================] - 116s 172ms/step - loss: 0.0056 - mae: 0.0491 - mse: 0.0056 - val_loss: 0.0126 - val_mae: 0.0770 - val_mse: 0.0126
Epoch 4/100
673/673 [==============================] - 114s 169ms/step - loss: 0.0053 - mae: 0.0478 - mse: 0.0053 - val_loss: 0.0136 - val_mae: 0.0819 - val_mse: 0.0136
Epoch 5/100
673/673 [==============================] - 115s 170ms/step - loss: 0.0050 - mae: 0.0466 - mse: 0.0050 - val_loss: 0.0136 - val_mae: 0.0812 - val_mse: 0.0136
Epoch 6/100
673/673 [==============================] - 113s 168ms/step - loss: 0.0048 - mae: 0.0456 - mse: 0.0048 - val_loss: 0.0123 - val_mae: 0.0760 - va

In [40]:
dataset_version = 'deep4pc_test1'
model_version = 'model_' + dataset_version + '_5_patience20'

createFolder('./Test/' + model_version)
f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version + '/' + model_version)
#model.summary()
outputs = model(x_test[0:10])

print(outputs)
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 10) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 23, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 10, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 4, 6, 32)        

## dataset : 'deep4pc_test1' + 'deep4pc_test2_existing'
### Epoch 28/100 38/38 [==============================] - 6s 165ms/step - loss: 0.0029 - mae: 0.0324 - mse: 0.0029 - val_loss: 0.0038 - val_mae: 0.0358 - val_mse: 0.0038


In [7]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataset_version = 'deep4pc_test1'
dataset_version_existing = 'deep4pc_test2_existing'
model_version = 'model_' + dataset_version_existing + '_5_patience20'

createFolder('./Test/' + model_version)
f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
with h5py.File('./Model/Model_' + dataset_version_existing + '/dataset_hsv_' + dataset_version_existing + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version_existing + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version_existing + '/' + model_version)
#model.summary()
outputs = model(x_test[0:10])

print(outputs)
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 10) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 6, 32)         0

2022-02-07 16:34:46.674535: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vividlee/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-02-07 16:34:46.674635: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vividlee/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-02-07 16:34:46.674701: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vividlee/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-02-07 16:34:46.700481: W tensorflow/stream_ex

Epoch 1/100
38/38 [==============================] - 7s 168ms/step - loss: 0.0247 - mae: 0.1017 - mse: 0.0247 - val_loss: 0.0133 - val_mae: 0.0898 - val_mse: 0.0133
Epoch 2/100
38/38 [==============================] - 6s 163ms/step - loss: 0.0110 - mae: 0.0746 - mse: 0.0110 - val_loss: 0.0067 - val_mae: 0.0597 - val_mse: 0.0067
Epoch 3/100
38/38 [==============================] - 6s 162ms/step - loss: 0.0092 - mae: 0.0672 - mse: 0.0092 - val_loss: 0.0163 - val_mae: 0.1058 - val_mse: 0.0163
Epoch 4/100
38/38 [==============================] - 6s 161ms/step - loss: 0.0081 - mae: 0.0615 - mse: 0.0081 - val_loss: 0.0060 - val_mae: 0.0547 - val_mse: 0.0060
Epoch 5/100
38/38 [==============================] - 6s 161ms/step - loss: 0.0074 - mae: 0.0573 - mse: 0.0074 - val_loss: 0.0070 - val_mae: 0.0624 - val_mse: 0.0070
Epoch 6/100
38/38 [==============================] - 6s 160ms/step - loss: 0.0069 - mae: 0.0546 - mse: 0.0069 - val_loss: 0.0060 - val_mae: 0.0500 - val_mse: 0.0060
Epoch 7/10

2022-02-07 16:37:45.210683: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./Model/Model_deep4pc_test2_existing/model_deep4pc_test2_existing_5_patience20/assets
tf.Tensor(
[[0.90058583 0.22449778]
 [0.9784754  0.22367086]
 [1.0318474  0.20568928]
 [0.8807407  0.21662639]
 [0.94416195 0.19423714]
 [0.8018071  0.20891562]
 [0.77657753 0.19982907]
 [0.7426489  0.20155053]
 [0.6980996  0.19960107]
 [0.90882    0.19041562]], shape=(10, 2), dtype=float32)


In [8]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataset_version = 'deep4pc_test1'
dataset_version_existing = 'deep4pc_test2_existing'
model_version = 'model_' + dataset_version_existing + '_6_patience5'

createFolder('./Test/' + model_version)
f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
with h5py.File('./Model/Model_' + dataset_version_existing + '/dataset_hsv_' + dataset_version_existing + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version_existing + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version_existing + '/' + model_version)
#model.summary()
outputs = model(x_test[0:10])

print(outputs)
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 10) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 23, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 10, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 4, 6, 32)        

In [9]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataset_version = 'deep4pc_test1'
dataset_version_existing = 'deep4pc_test2_existing'
model_version = 'model_' + dataset_version_existing + '_7_patience8'

createFolder('./Test/' + model_version)
f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
with h5py.File('./Model/Model_' + dataset_version_existing + '/dataset_hsv_' + dataset_version_existing + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version_existing + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version_existing + '/' + model_version)
#model.summary()
outputs = model(x_test[0:10])

print(outputs)
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 10) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 23, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 10, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 6, 32)        

In [10]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataset_version = 'deep4pc_test1'
dataset_version_existing = 'deep4pc_test2_existing'
model_version = 'model_' + dataset_version_existing + '_8_patience10'

createFolder('./Test/' + model_version)
f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
with h5py.File('./Model/Model_' + dataset_version_existing + '/dataset_hsv_' + dataset_version_existing + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version_existing + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version_existing + '/' + model_version)
#model.summary()
outputs = model(x_test[0:10])

print(outputs)
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 10) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 23, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 10, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 4, 6, 32)        

In [11]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataset_version = 'deep4pc_test1'
dataset_version_existing = 'deep4pc_test2_existing'
model_version = 'model_' + dataset_version_existing + '_9_patience12'

createFolder('./Test/' + model_version)
f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
with h5py.File('./Model/Model_' + dataset_version_existing + '/dataset_hsv_' + dataset_version_existing + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=12)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version_existing + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version_existing + '/' + model_version)
#model.summary()
outputs = model(x_test[0:20])

print(outputs)
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 20) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 23, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 10, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 4, 6, 32)        

In [12]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataset_version = 'deep4pc_test1'
dataset_version_existing = 'deep4pc_test2_existing'
model_version = 'model_' + dataset_version_existing + '_10_patience15'

createFolder('./Test/' + model_version)
f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
with h5py.File('./Model/Model_' + dataset_version_existing + '/dataset_hsv_' + dataset_version_existing + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])

    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version_existing + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version_existing + '/' + model_version)
#model.summary()
outputs = model(x_test[0:20])

print(outputs)
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 20) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 23, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 10, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 4, 6, 32)        

In [13]:
# one final HDF5

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataset_version = 'deep4pc_test3_all'
#dataset_version_existing = 'deep4pc_test2_existing'
model_version = 'model_' + dataset_version + '_1_patience3'

createFolder('./Test/' + model_version)
f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version+ '/' + model_version)
#model.summary()
outputs = model(x_test[0:20])

print(outputs)
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 20) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 23, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 10, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 4, 6, 32)        

2022-02-07 18:46:08.306570: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6699405312 exceeds 10% of free system memory.


710/710 [==============================] - 117s 165ms/step - loss: 0.0081 - mae: 0.0584 - mse: 0.0081 - val_loss: 0.0111 - val_mae: 0.0716 - val_mse: 0.0111
Epoch 2/100
710/710 [==============================] - 116s 164ms/step - loss: 0.0061 - mae: 0.0504 - mse: 0.0061 - val_loss: 0.0115 - val_mae: 0.0720 - val_mse: 0.0115
Epoch 3/100
710/710 [==============================] - 117s 164ms/step - loss: 0.0056 - mae: 0.0487 - mse: 0.0056 - val_loss: 0.0118 - val_mae: 0.0743 - val_mse: 0.0118
Epoch 4/100
710/710 [==============================] - 117s 165ms/step - loss: 0.0053 - mae: 0.0474 - mse: 0.0053 - val_loss: 0.0114 - val_mae: 0.0729 - val_mse: 0.0114


1426/1426 [==============================] - 8s 6ms/step - loss: 0.0114 - mae: 0.0729 - mse: 0.0114
[0.011382139287889004, 0.07289628684520721, 0.011382139287889004]
INFO:tensorflow:Assets written to: ./Model/Model_deep4pc_test3_all/model_deep4pc_test3_all_1_patience3/assets
tf.Tensor(
[[0.45389885 0.22172397]
 [0.4535658  0.2218578

In [14]:
# one final HDF5

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataset_version = 'deep4pc_test3_all'
#dataset_version_existing = 'deep4pc_test2_existing'
model_version = 'model_' + dataset_version + '_2_patience6'

createFolder('./Test/' + model_version)
# f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version+ '/' + model_version)
#model.summary()
outputs = model(x_test[0:20])

print(outputs)

f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 20) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 23, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 10, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 4, 6, 32)        

2022-02-07 19:00:33.412895: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6699405312 exceeds 10% of free system memory.


710/710 [==============================] - 119s 167ms/step - loss: 0.0084 - mae: 0.0594 - mse: 0.0084 - val_loss: 0.0119 - val_mae: 0.0742 - val_mse: 0.0119
Epoch 2/100
710/710 [==============================] - 118s 166ms/step - loss: 0.0062 - mae: 0.0508 - mse: 0.0062 - val_loss: 0.0121 - val_mae: 0.0742 - val_mse: 0.0121
Epoch 3/100
710/710 [==============================] - 117s 164ms/step - loss: 0.0057 - mae: 0.0489 - mse: 0.0057 - val_loss: 0.0132 - val_mae: 0.0782 - val_mse: 0.0132
Epoch 4/100
710/710 [==============================] - 117s 164ms/step - loss: 0.0053 - mae: 0.0475 - mse: 0.0053 - val_loss: 0.0119 - val_mae: 0.0742 - val_mse: 0.0119
Epoch 5/100
710/710 [==============================] - 117s 165ms/step - loss: 0.0050 - mae: 0.0463 - mse: 0.0050 - val_loss: 0.0116 - val_mae: 0.0726 - val_mse: 0.0116
Epoch 6/100
710/710 [==============================] - 117s 165ms/step - loss: 0.0048 - mae: 0.0453 - mse: 0.0048 - val_loss: 0.0126 - val_mae: 0.0771 - val_mse: 0.012

In [3]:
# one final HDF5

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataset_version = 'deep4pc_test3_all'
#dataset_version_existing = 'deep4pc_test2_existing'
model_version = 'model_' + dataset_version + '_3_patience9'

createFolder('./Test/' + model_version)
# f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=9)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version+ '/' + model_version)
#model.summary()
outputs = model(x_test[0:20])

print(outputs)

f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 20) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


2022-02-08 13:54:48.963393: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vividlee/anaconda3/envs/py38/lib/python3.8/site-packages/cv2/../../lib64:
2022-02-08 13:54:48.963505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vividlee/anaconda3/envs/py38/lib/python3.8/site-packages/cv2/../../lib64:
2022-02-08 13:54:48.963582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vividlee/anaconda3/envs/py38/lib/python3.8/site-packages/cv2/../../lib64:
2022-02-08 13:54:48.

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 6, 32)         0

2022-02-08 13:54:49.075151: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6699405312 exceeds 10% of free system memory.


Epoch 1/100
710/710 [==============================] - 120s 167ms/step - loss: 0.0081 - mae: 0.0586 - mse: 0.0081 - val_loss: 0.0117 - val_mae: 0.0739 - val_mse: 0.0117
Epoch 2/100
710/710 [==============================] - 119s 168ms/step - loss: 0.0061 - mae: 0.0507 - mse: 0.0061 - val_loss: 0.0128 - val_mae: 0.0810 - val_mse: 0.0128
Epoch 3/100
710/710 [==============================] - 119s 168ms/step - loss: 0.0057 - mae: 0.0490 - mse: 0.0057 - val_loss: 0.0114 - val_mae: 0.0730 - val_mse: 0.0114
Epoch 4/100
710/710 [==============================] - 117s 164ms/step - loss: 0.0053 - mae: 0.0477 - mse: 0.0053 - val_loss: 0.0119 - val_mae: 0.0763 - val_mse: 0.0119
Epoch 5/100
710/710 [==============================] - 117s 165ms/step - loss: 0.0051 - mae: 0.0465 - mse: 0.0051 - val_loss: 0.0118 - val_mae: 0.0749 - val_mse: 0.0118
Epoch 6/100
710/710 [==============================] - 119s 167ms/step - loss: 0.0048 - mae: 0.0456 - mse: 0.0048 - val_loss: 0.0119 - val_mae: 0.0751 - va

2022-02-08 14:18:37.646012: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./Model/Model_deep4pc_test3_all/model_deep4pc_test3_all_3_patience9/assets
tf.Tensor(
[[0.40231705 0.22680396]
 [0.40230942 0.22686745]
 [0.40195498 0.22650763]
 [0.41782853 0.23110434]
 [0.4265459  0.22921488]
 [0.44574454 0.2241148 ]
 [0.4419746  0.22081223]
 [0.41891798 0.22934283]
 [0.41181752 0.22817841]
 [0.42935726 0.22290933]
 [0.45981333 0.21819475]
 [0.4414697  0.22807834]
 [0.44159728 0.2280785 ]
 [0.44966063 0.2302664 ]
 [0.39477286 0.22627959]
 [0.4053612  0.22730774]
 [0.42451656 0.23390462]
 [0.42450696 0.23403156]
 [0.37123662 0.2318321 ]
 [0.37097827 0.23160622]], shape=(20, 2), dtype=float32)


In [4]:
# one final HDF5

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataset_version = 'deep4pc_test3_all'
#dataset_version_existing = 'deep4pc_test2_existing'
model_version = 'model_' + dataset_version + '_4_patience12'

createFolder('./Test/' + model_version)
# f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=12)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version+ '/' + model_version)
#model.summary()
outputs = model(x_test[0:20])

print(outputs)

f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 20) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 23, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 10, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 4, 6, 32)        

2022-02-08 14:25:30.609703: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6699405312 exceeds 10% of free system memory.


Epoch 1/100
710/710 [==============================] - 120s 168ms/step - loss: 0.0081 - mae: 0.0591 - mse: 0.0081 - val_loss: 0.0114 - val_mae: 0.0759 - val_mse: 0.0114
Epoch 2/100
710/710 [==============================] - 118s 166ms/step - loss: 0.0062 - mae: 0.0509 - mse: 0.0062 - val_loss: 0.0118 - val_mae: 0.0741 - val_mse: 0.0118
Epoch 3/100
710/710 [==============================] - 118s 166ms/step - loss: 0.0057 - mae: 0.0491 - mse: 0.0057 - val_loss: 0.0111 - val_mae: 0.0721 - val_mse: 0.0111
Epoch 4/100
710/710 [==============================] - 120s 169ms/step - loss: 0.0053 - mae: 0.0477 - mse: 0.0053 - val_loss: 0.0112 - val_mae: 0.0728 - val_mse: 0.0112
Epoch 5/100
710/710 [==============================] - 119s 167ms/step - loss: 0.0051 - mae: 0.0466 - mse: 0.0051 - val_loss: 0.0115 - val_mae: 0.0732 - val_mse: 0.0115
Epoch 6/100
710/710 [==============================] - 117s 165ms/step - loss: 0.0048 - mae: 0.0456 - mse: 0.0048 - val_loss: 0.0118 - val_mae: 0.0736 - va

In [5]:
# one final HDF5

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

dataset_version = 'deep4pc_test3_all'
#dataset_version_existing = 'deep4pc_test2_existing'
model_version = 'model_' + dataset_version + '_5_patience15'

createFolder('./Test/' + model_version)
# f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')

with h5py.File('./Model/Model_' + dataset_version + '/dataset_hsv_' + dataset_version + '.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[callback])

print("\n")
print(model.evaluate(x_test, y_test))


# model.save('./script/Model/'+model_version)
tf.saved_model.save(model, './Model/Model_' + dataset_version + '/' + model_version)

del model

#model = tf.keras.models.load_model('./script/Model/'+model_version)
#print(model.evaluate(x_test, y_test))
#model.summary()

model = tf.saved_model.load('./Model/Model_' + dataset_version+ '/' + model_version)
#model.summary()
outputs = model(x_test[0:20])

print(outputs)

f = open('./Test/' + model_version + '/' + model_version + '_test.txt', 'a')
f.write("{}".format(outputs))

x_test *= 255
    
for i in range (0, 20) :
    cv2.imwrite('./Test/' + model_version + '/'+ model_version + '_' + str(i) + '.jpeg', x_test[i])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 46, 62, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 23, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 21, 29, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 10, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 8, 12, 32)         9248      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 6, 32)        

2022-02-08 14:56:17.054262: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6699405312 exceeds 10% of free system memory.


Epoch 1/100
710/710 [==============================] - 123s 173ms/step - loss: 0.0079 - mae: 0.0585 - mse: 0.0079 - val_loss: 0.0125 - val_mae: 0.0774 - val_mse: 0.0125
Epoch 2/100
710/710 [==============================] - 121s 170ms/step - loss: 0.0061 - mae: 0.0508 - mse: 0.0061 - val_loss: 0.0118 - val_mae: 0.0737 - val_mse: 0.0118
Epoch 3/100
710/710 [==============================] - 117s 165ms/step - loss: 0.0057 - mae: 0.0491 - mse: 0.0057 - val_loss: 0.0119 - val_mae: 0.0744 - val_mse: 0.0119
Epoch 4/100
710/710 [==============================] - 118s 167ms/step - loss: 0.0053 - mae: 0.0476 - mse: 0.0053 - val_loss: 0.0114 - val_mae: 0.0738 - val_mse: 0.0114
Epoch 5/100
710/710 [==============================] - 117s 164ms/step - loss: 0.0050 - mae: 0.0465 - mse: 0.0050 - val_loss: 0.0118 - val_mae: 0.0739 - val_mse: 0.0118
Epoch 6/100
710/710 [==============================] - 117s 165ms/step - loss: 0.0048 - mae: 0.0455 - mse: 0.0048 - val_loss: 0.0114 - val_mae: 0.0759 - va